**Observation about the data**

This is real world data that is messy, which most YouTube video's don't even bother to look at, or just give it a cursory glance. That unsexy part of trying to fix data gets glossed over or outright ignored. Fun? No. Necessary? Yes.

To do due it justice won't fit nicely into a 1/2 hour YouTube video about EDA or datascience. So, I'll make an attempt to fix the most glaring issues. Real world dictates progress, not perfection. And nothing is ever perfect anyways. Models are just that. Models of reality, not reality itself.

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Scott\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
plt.style.use('ggplot')
pd.options.display.width = 0

## Import Data

In [2]:
train = pd.read_excel('../Titanic_EDA/titanic/train_fix.xlsx')
test = pd.read_csv('../Titanic_EDA/titanic/test.csv')

In [3]:
train.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Boarding,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5,21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC,17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2.,3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,NaN,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,NaN,373450,8.0500,NaN,S
5,6,0,3,male,NaN,0,0,NaN,330877,8.4583,NaN,Q
6,7,0,1,male,54.0,0,0,NaN,17463,51.8625,E46,S
7,8,0,3,male,2.0,3,1,NaN,349909,21.0750,NaN,S
8,9,1,3,female,27.0,0,2,NaN,347742,11.1333,NaN,S
9,10,1,2,female,14.0,1,0,NaN,237736,30.0708,NaN,C


Some things are just more convenient to do on a spreadsheet. I removed the passenger names column. The *Ticket* column had what research shows as a ticket number along with Deck info and maybe a embarking/ boarding order number, so I separated that into two separate colulmns.

- Need to change *Sex* column from male/female to 1/0, and estimate missing ages.

In [4]:
train = pd.DataFrame(train)

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       230 non-null    object 
 8   Boarding     886 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket         661
Boarding         5
Fare             0
Cabin          687
Embarked         2
dtype: int64

It looks like many ages are missing, and per the Data Description, "**age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5"**, so need to imput those rows, as there are far too many to delete, unlike Embarked, where only two are missing. My guess is that those individuals were very lucky to have missed the boat. As for *Cabin* hmm, might have to do some feature engineering or some sort to see if any possible loose correlations exist to be able to assign a cabin to all those NaN's.
- A lot of missing *Ticket* info and *Cabin* info that will need to be imputed.
- Not to certain the *Boarding* column will be of much use, but I'll keep it around just in case.

## Modifing DataFrame

In [8]:
# Convert 'Sex' column: 'male' -> 1, 'female' -> 0
train['Sex'] = train['Sex'].map({'male': 1, 'female': 0}).astype('int32')

In [9]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Boarding,Fare,Cabin,Embarked
0,1,0,3,1,22.0,1,0,A/5,21171,7.2500,NaN,S
1,2,1,1,0,38.0,1,0,PC,17599,71.2833,C85,C
2,3,1,3,0,26.0,0,0,STON/O2.,3101282,7.9250,NaN,S
3,4,1,1,0,35.0,1,0,NaN,113803,53.1000,C123,S
4,5,0,3,1,35.0,0,0,NaN,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.0,0,0,NaN,211536,13.0000,NaN,S
887,888,1,1,0,19.0,0,0,NaN,112053,30.0000,B42,S
888,889,0,3,0,NaN,1,2,W./C.,6607,23.4500,NaN,S
889,890,1,1,1,26.0,0,0,NaN,111369,30.0000,C148,C


Added missing age's using sci-kit learn Linear Regression for Imputation. Using regular mean imputation is the lazy/bad way to input missing data. It can alter the data's kurtosis by reducing variability by increasing the 'peakedness' or flatten the tails.

In [10]:
#Features to use to computate missing age values.
features = ['Sex', 'Pclass', 'Parch', 'SibSp']

# Split the data into observed and missing Age.
train_obs = train[train['Age'].notnull()]
train_missing = train[train['Age'].isnull()]

# Fit the model on rows with observed Age.
X_train = train_obs[features]
y_train = train_obs['Age']
lr = LinearRegression().fit(X_train, y_train)

# Predict Age for missing entries.
X_missing = train_missing[features]
predicted_age = lr.predict(X_missing)

# add noise to reflect uncertainty.
sigma = np.std(y_train - lr.predict(X_train))
predicted_age_noisy = predicted_age + np.random.normal(0, sigma, size=predicted_age.shape)

# Impute the missing Age values.
train.loc[train['Age'].isnull(), 'Age'] = predicted_age_noisy

In [11]:
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Boarding,Fare,Cabin,Embarked
0,1,0,3,1,22.000000,1,0,A/5,21171,7.2500,NaN,S
1,2,1,1,0,38.000000,1,0,PC,17599,71.2833,C85,C
2,3,1,3,0,26.000000,0,0,STON/O2.,3101282,7.9250,NaN,S
3,4,1,1,0,35.000000,1,0,NaN,113803,53.1000,C123,S
4,5,0,3,1,35.000000,0,0,NaN,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,NaN,211536,13.0000,NaN,S
887,888,1,1,0,19.000000,0,0,NaN,112053,30.0000,B42,S
888,889,0,3,0,16.961247,1,2,W./C.,6607,23.4500,NaN,S
889,890,1,1,1,26.000000,0,0,NaN,111369,30.0000,C148,C


Here I decided to use Mode Imputation as I discovered that the first letter in *Cabin* refers to the deck, and making an assumption that those with the money had the ability to purchase a cabin, and those without, didn't.

In [12]:
# Extract the first letter of the Cabin column (deck identifier)
train['Deck'] = train['Cabin'].str[0]

# Impute missing Deck values with the most common Deck per Pclass
train['Deck'] = train.groupby('Pclass')['Deck'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'U'))


In [13]:
print(train[['Pclass', 'Deck']].head(10))  # Check first 10 rows

   Pclass Deck
0       3    F
1       1    C
2       3    F
3       1    C
4       3    F
5       3    F
6       1    E
7       3    F
8       3    F
9       2    F


In [14]:
#Added new Deck column
train

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Boarding,Fare,Cabin,Embarked,Deck
0,1,0,3,1,22.000000,1,0,A/5,21171,7.2500,NaN,S,F
1,2,1,1,0,38.000000,1,0,PC,17599,71.2833,C85,C,C
2,3,1,3,0,26.000000,0,0,STON/O2.,3101282,7.9250,NaN,S,F
3,4,1,1,0,35.000000,1,0,NaN,113803,53.1000,C123,S,C
4,5,0,3,1,35.000000,0,0,NaN,373450,8.0500,NaN,S,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,NaN,211536,13.0000,NaN,S,F
887,888,1,1,0,19.000000,0,0,NaN,112053,30.0000,B42,S,B
888,889,0,3,0,16.961247,1,2,W./C.,6607,23.4500,NaN,S,F
889,890,1,1,1,26.000000,0,0,NaN,111369,30.0000,C148,C,C


Now to change the *Deck* letter to number via one-hot-encoding. 

In [15]:
train_encoded = pd.get_dummies(train['Deck'], dtype=int)
train_encoded = train_encoded.rename(columns={'A': 'Deck-A', 'B': 'Deck-B',  'C': 'Deck-C', 'D': 'Deck-D', 'E': 'Deck-E', 'F': 'Deck-F',  'G': 'Deck-G', 'T': 'Deck-T'})
print('\n DataFrame after performing One-hot Encoding')
display(train_encoded)


 DataFrame after performing One-hot Encoding


,Deck-A,Deck-B,Deck-C,Deck-D,Deck-E,Deck-F,Deck-G,Deck-T
0,0,0,0,0,0,1,0,0
1,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0
3,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
886,0,0,0,0,0,1,0,0
887,0,1,0,0,0,0,0,0
888,0,0,0,0,0,1,0,0
889,0,0,1,0,0,0,0,0


Lets do the same for *Embarked* column

In [16]:
# Extract the first letter of the Embarked column
train['Embarked'] = train['Embarked'].str[0]

# Impute missing Embarked values with the most common per Pclass
train['Embarked'] = train.groupby('Pclass')['Embarked'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'U'))

In [18]:
print(train[['Pclass', 'Embarked']].head(10))  # Check first 10 rows

   Pclass Embarked
0       3        S
1       1        C
2       3        S
3       1        S
4       3        S
5       3        Q
6       1        S
7       3        S
8       3        S
9       2        C


In [22]:
train_embarked = pd.get_dummies(train['Embarked'], dtype=int)
train_embarked = train_embarked.rename(columns={'C': 'Embarked-C', 'Q': 'Embarked-Q',  'S': 'Embarked-S'})
print('\n DataFrame after performing One-hot Encoding')
display(train_embarked)


 DataFrame after performing One-hot Encoding


,Embarked-C,Embarked-Q,Embarked-S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
886,0,0,1
887,0,0,1
888,0,0,1
889,1,0,0


Add the new one-hot encoded *Deck* and *Embarked* back into the DataFrame

In [ ]:

train_new = train.copy()
train_new[['Deck-A', 'Deck-B', 'Deck-C', 'Deck-D', 'Deck-E', 'Deck-F', 'Deck-G', 'Deck-T']] = train_encoded[['Deck-A', 'Deck-B', 'Deck-C', 'Deck-D', 'Deck-E', 'Deck-F', 'Deck-G', 'Deck-T']]
print(train_new)

     PassengerId  Survived  Pclass  Sex        Age  SibSp  Parch    Ticket  \
0              1         0       3    1  22.000000      1      0       A/5   
1              2         1       1    0  38.000000      1      0        PC   
2              3         1       3    0  26.000000      0      0  STON/O2.   
3              4         1       1    0  35.000000      1      0       NaN   
4              5         0       3    1  35.000000      0      0       NaN   
..           ...       ...     ...  ...        ...    ...    ...       ...   
886          887         0       2    1  27.000000      0      0       NaN   
887          888         1       1    0  19.000000      0      0       NaN   
888          889         0       3    0  16.961247      1      2     W./C.   
889          890         1       1    1  26.000000      0      0       NaN   
890          891         0       3    1  32.000000      0      0       NaN   

    Boarding     Fare  ... Embarked Deck Deck-A  Deck-B  Deck-C

In [28]:
train_emb = train_new.copy()
train_emb[['Embarked-C', 'Embarked-Q', 'Embarked-S']] = train_embarked[['Embarked-C', 'Embarked-Q', 'Embarked-S']]
print(train_emb)

     PassengerId  Survived  Pclass  Sex        Age  SibSp  Parch    Ticket  \
0              1         0       3    1  22.000000      1      0       A/5   
1              2         1       1    0  38.000000      1      0        PC   
2              3         1       3    0  26.000000      0      0  STON/O2.   
3              4         1       1    0  35.000000      1      0       NaN   
4              5         0       3    1  35.000000      0      0       NaN   
..           ...       ...     ...  ...        ...    ...    ...       ...   
886          887         0       2    1  27.000000      0      0       NaN   
887          888         1       1    0  19.000000      0      0       NaN   
888          889         0       3    0  16.961247      1      2     W./C.   
889          890         1       1    1  26.000000      0      0       NaN   
890          891         0       3    1  32.000000      0      0       NaN   

    Boarding     Fare  ... Deck-B Deck-C Deck-D  Deck-E  Deck-F

In [29]:
train_combined = pd.concat([train_embarked[['Embarked-C', 'Embarked-Q', 'Embarked-S']], 
                            train_encoded[['Deck-A', 'Deck-B', 'Deck-C', 'Deck-D', 'Deck-E', 'Deck-F', 'Deck-G', 'Deck-T']]], 
                           axis=1)


In [30]:
train_combined

,Embarked-C,Embarked-Q,Embarked-S,Deck-A,Deck-B,Deck-C,Deck-D,Deck-E,Deck-F,Deck-G,Deck-T
0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0,0,1,0,0
3,0,0,1,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,0,1,0,0,0,0,0,1,0,0
887,0,0,1,0,1,0,0,0,0,0,0
888,0,0,1,0,0,0,0,0,1,0,0
889,1,0,0,0,0,1,0,0,0,0,0


In [31]:
train_final = pd.concat([train, train_combined], axis=1)
train_final

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Boarding,Fare,...,Embarked-Q,Embarked-S,Deck-A,Deck-B,Deck-C,Deck-D,Deck-E,Deck-F,Deck-G,Deck-T
0,1,0,3,1,22.000000,1,0,A/5,21171,7.2500,...,0,1,0,0,0,0,0,1,0,0
1,2,1,1,0,38.000000,1,0,PC,17599,71.2833,...,0,0,0,0,1,0,0,0,0,0
2,3,1,3,0,26.000000,0,0,STON/O2.,3101282,7.9250,...,0,1,0,0,0,0,0,1,0,0
3,4,1,1,0,35.000000,1,0,NaN,113803,53.1000,...,0,1,0,0,1,0,0,0,0,0
4,5,0,3,1,35.000000,0,0,NaN,373450,8.0500,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,1,27.000000,0,0,NaN,211536,13.0000,...,0,1,0,0,0,0,0,1,0,0
887,888,1,1,0,19.000000,0,0,NaN,112053,30.0000,...,0,1,0,1,0,0,0,0,0,0
888,889,0,3,0,16.961247,1,2,W./C.,6607,23.4500,...,0,1,0,0,0,0,0,1,0,0
889,890,1,1,1,26.000000,0,0,NaN,111369,30.0000,...,0,0,0,0,1,0,0,0,0,0


*Ticket* column has so many missing values now since extracting the boarding/embarkation numbers to a new column and have Embarked and Deck columns with both categorical and one-hot encoded columns, imo there isn't much value left in the Ticket column.